In [ ]:
 
from pprint import pprint
from urllib.parse import urljoin
from requests.exceptions import RequestException
import requests
from datetime import datetime
import time
import os
import lxml
from memorious.operations.extract import extract_zip
from openpyxl import load_workbook, Workbook

from concurrent.futures import ThreadPoolExecutor
#from part_test import download_and_process, process_zip_file, preprocess_json, process_extracted_files
import time
timestamp = int(time.time())  # Add a timestamp for uniqueness


In [9]:
# Import from ua_kmr_voting_excel_functions.py function find_last_row
from ua_kmr_voting_excel_functions import find_last_row, request_page, count_non_empty_columns, transform_results

In [ ]:
# Counter to ensure uniqueness
counter = 0

# Adjust this as needed, depending on the number of threads you want to use
MAX_WORKERS = 4
zipfile_paths = []
CHUNK_SIZE = 100 
URL_FULL = "https://kmr.gov.ua/uk/result_golosuvanya?title=&field_start_date_n_h_value%5Bmin%5D=&field_start_date_n_h_value%5Bmax%5D=&page="
URL = 'https://kmr.gov.ua/uk/result_golosuvanya?title=&field_start_date_n_h_value[min]&field_start_date_n_h_value[max]&page=0&start=0'

HEADERS_XLSX = ["id",  "time", "question", "status", "short_name", "result", "file_name", "source_url"]

In [ ]:
### MAIN FUNCTION ###
### PROCESS XLSX FILES ###
processed_files = set()

# Create a new folder for the processed files
if not os.path.exists("processed_files"):
    os.makedirs("processed_files")

for i in range(15, 22): # Range adjusted to match the number of pages where XLSX files are available
#for i in range(16, 21): # Range adjusted to match the number of pages where XLSX files are available
#for i in range(1, 2):
    URL_FULL_PAGE = URL_FULL + str(i)
    response_main_page = requests.get(URL_FULL_PAGE)
    doc = lxml.html.fromstring(response_main_page.text)
    block = doc.xpath('//div[@class="view-content"]')

    for item in block[:1]:
        for j in item.getchildren():
            #print(j.text_content())
            # if 'zip' in j.text_content():  # ZIP files
            #     link = j.xpath('.//a/@href')[1]
            #     print("\n")
            #     context.log.info(f"Processing ZIP file: {link}")
            #     executor.submit(download_and_process, context, link, True) # Download and process ZIP files

            if 'xlsx' in j.text_content():  # XLSX files
                link_xlsx = j.xpath('.//a/@href')[1]
                print("\n", link_xlsx)
                #context.log.info(f"Processing XLSX file: {link_xlsx}")
                #download_and_process(link_xlsx, False)

                doc_response = requests.get(link_xlsx)
                print(doc_response)
                if doc_response.status_code == 200:

                    file_name = link_xlsx.split('/')[-1]
                    file_path = f"processed_files/temp_{file_name}"

                    print(f"Saving file: {file_path}")

                    with open(file_path, "wb") as f:
                        f.write(doc_response.content)





 https://kmr.gov.ua/sites/default/files/22.07.2021_struk.xlsx
<Response [200]>
Saving file: processed_files/temp_22.07.2021_struk.xlsx

 https://kmr.gov.ua/sites/default/files/08.07.2021_struk.xlsx
<Response [200]>
Saving file: processed_files/temp_08.07.2021_struk.xlsx

 https://kmr.gov.ua/sites/default/files/15_24.06.2021.xlsx
<Response [200]>
Saving file: processed_files/temp_15_24.06.2021.xlsx

 https://kmr.gov.ua/sites/default/files/14_10.06.2021_0.xlsx
<Response [200]>
Saving file: processed_files/temp_14_10.06.2021_0.xlsx

 https://kmr.gov.ua/sites/default/files/13_27.05.2021.xlsx
<Response [200]>
Saving file: processed_files/temp_13_27.05.2021.xlsx

 https://kmr.gov.ua/sites/default/files/12_13.05.2021.xlsx
<Response [200]>
Saving file: processed_files/temp_12_13.05.2021.xlsx

 https://kmr.gov.ua/sites/default/files/11_22.04.2021.xlsx
<Response [200]>
Saving file: processed_files/temp_11_22.04.2021.xlsx

 https://kmr.gov.ua/sites/default/files/10_08.04.2021.xlsx
<Response [200

In [3]:
class Context:
	def __init__(self, work_path):
		self.work_path = work_path

context = Context(work_path='/Users/Oksana/Documents/PERSONAL_PRJCTS/ua_kmr_voting/src/processed_files')

xlsx_files = [file for file in os.listdir(context.work_path) if file.endswith('.xlsx')]
xlsx_files

['temp_3_14.12.2020.xlsx',
 'temp_08.07.xlsx',
 'temp_13_27.05.2021.xlsx',
 'temp_4_22.12.2020.xlsx',
 'temp_7_23.02.2021.xlsx',
 'temp_11_22.04.2021.xlsx',
 'temp_5_24.12.2020.xlsx',
 '1_03.12.2020.xlsx',
 'temp_15_24.06.2021.xlsx',
 'temp_6_11.02.2021.xlsx',
 'temp_10_08.04.2021.xlsx',
 'temp_8_04.03.2021.xlsx',
 'temp_12_13.05.2021.xlsx',
 'temp_2_08.12.2020.xlsx',
 'temp_14_10.06.2021_0.xlsx',
 'temp_9_11.03.2021.xlsx',
 'temp_22.07.xlsx']

In [30]:
import hashlib
from collections import Counter

counter = 0

#counter = Counter("")
def generate_deputy_id(deputy_name):
    """Generate a consistent ID for deputy names"""
    if not deputy_name:
        return None
    # Generate a unique ID based on a counter
    global counter
    counter += 1
    return str(counter)


In [31]:

def store_excel(context, data):
    
    # Generate deputy ID from deputy name
    deputy_name = data.get('entity_name')
    deputy_id = generate_deputy_id(deputy_name)
    
    # Document data
    doc_data = {
        'Num_Question': data.get('fixed_col_1'),  # Original ID for reference
        'GLTime': str(data.get('fixed_col_2')), 
        'GL_Text': data.get('fixed_col_3'), 
        'RESULT': data.get('fixed_col_4'),  
        'DPName': deputy_name,  
        'id_original': deputy_id,  # New field for deputy ID
        'DPGolos': data.get('result'), 
        "file_name": data.get('xlsx_file'),
        'retrieved_at': datetime.now().isoformat()
    }
    
    # Establish connection to SQLite database
    conn = sqlite3.connect('voting_excel_data2.db')
    cursor = conn.cursor()

    # Ensure the table exists (CREATE IF NOT EXISTS) - Updated with id_original field
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS ua_kmr_voting_xlsx (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Num_Question TEXT NOT NULL,
        GLTime TEXT,
        GL_Text TEXT,
        RESULT TEXT,
        DPName TEXT,
        id_original TEXT,
        DPGolos TEXT,
        file_name TEXT,
        retrieved_at TEXT,
        source_url TEXT,
        UNIQUE(Num_Question, GLTime, GL_Text, RESULT, DPName, DPGolos, file_name)
    );
    """
    cursor.execute(create_table_sql)
    
    # Check if the record exists based on the unique fields
    check_record_sql = """
    SELECT * FROM ua_kmr_voting_xlsx WHERE Num_Question = ? AND GLTime = ? AND GL_Text = ? 
          AND RESULT = ? AND DPName = ? AND DPGolos = ? AND file_name = ?;
    """
    cursor.execute(check_record_sql, (doc_data['Num_Question'], doc_data['GLTime'], doc_data['GL_Text'], 
                                      doc_data['RESULT'], doc_data['DPName'], doc_data['DPGolos'], doc_data['file_name']))
    existing_record = cursor.fetchone()
    
    if existing_record:
        # If record exists, update it (including id_original)
        update_sql = """
        UPDATE ua_kmr_voting_xlsx
        SET GLTime = ?, GL_Text = ?, RESULT = ?, DPName = ?, id_original = ?, DPGolos = ?, retrieved_at = ?
        WHERE Num_Question = ? AND file_name = ?;
        """
        cursor.execute(update_sql, (
            doc_data['GLTime'], doc_data['GL_Text'], doc_data['RESULT'], doc_data['DPName'], 
            doc_data['id_original'], doc_data['DPGolos'], doc_data['retrieved_at'], 
            doc_data['Num_Question'], doc_data['file_name']
        ))
    else:
        # If record doesn't exist, insert a new one (including id_original)
        insert_sql = """
        INSERT INTO ua_kmr_voting_xlsx (Num_Question, GLTime, GL_Text, RESULT, DPName, id_original, DPGolos, file_name, retrieved_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        cursor.execute(insert_sql, (
            doc_data['Num_Question'], doc_data['GLTime'], doc_data['GL_Text'], doc_data['RESULT'], 
            doc_data['DPName'], doc_data['id_original'], doc_data['DPGolos'], doc_data['file_name'], doc_data['retrieved_at']
        ))
    
    # Commit the transaction and close the connection
    conn.commit()
    conn.close()


In [ ]:
for xlsx_file in xlsx_files:
    #context.log.info(f"Processing XLSX file: {xlsx_file}")

    file_path = os.path.join(context.work_path, xlsx_file)
    wb = load_workbook(file_path, read_only=False)
    ws = wb[wb.sheetnames[0]]

    max_row_N = find_last_row(ws)
    non_empty_col_count = count_non_empty_columns(ws) # Count non-empty columns in the first row
    
    start_col = 5 if non_empty_col_count == 125 else 6 # Adjust start column based on the number of columns

    fixed_cols_range = range(1, start_col)

    for row in ws.iter_rows(min_row=2, max_row=max_row_N, values_only=True):
        fixed_data = {f"fixed_col_{col}": row[col - 1] for col in fixed_cols_range}
        for col in range(start_col, start_col + 122):  # Adjust end column based on the number of columns
            entity_name = ws.cell(row=1, column=col).value
            #deputy_id = generate_deputy_id(entity_name) # Added
            result = row[col - 1] 

            # Create a dictionary for the row
            new_row = {
                **fixed_data,
                "entity_name": entity_name,
                #"deputy_id": deputy_id,        # 🆕 Now includes the generated ID
                "result": result,
                "xlsx_file": xlsx_file # hash
            }
            print(new_row)
            store_excel(context, new_row)

{'fixed_col_1': 1, 'fixed_col_2': '14.12.20 12:21', 'fixed_col_3': 'Приняття порядку денного за основу', 'fixed_col_4': 'Рішення ПРИЙНЯТЕ', 'entity_name': 'Андронов В. Є.', 'result': 'За', 'xlsx_file': 'temp_3_14.12.2020.xlsx'}


IntegrityError: UNIQUE constraint failed: ua_kmr_voting_xlsx.Num_Question, ua_kmr_voting_xlsx.GLTime, ua_kmr_voting_xlsx.GL_Text, ua_kmr_voting_xlsx.RESULT, ua_kmr_voting_xlsx.DPName, ua_kmr_voting_xlsx.DPGolos, ua_kmr_voting_xlsx.file_name